In [ ]:
from bs4 import BeautifulSoup
import urllib.request as urllib2
from urllib.request import urlopen
import re
import csv
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import itertools


Schwab_BASE = 'https://www.schwabjobs.com'
PNC_BASE = 'https://www.indeed.com/viewjob?'

PNC_URL = "https://www.indeed.com/jobs?q=Pnc+Bank"
Schwab_URL = "https://www.schwabjobs.com/List/Custom/Schwab-Job-Category"

stateList = []

count = 1
BASE = 'https://www.indeed.com/'
FIRST_LISTING_PAGE = 'https://www.indeed.com/cmp/Pnc-Financial-Services-Group/jobs?clearPrefilter=1'
LISTING_URL_PREFIX = 'https://www.indeed.com/cmp/Pnc-Financial-Services-Group/jobs?start='
LISTING_URL_POSTFIX = 'clearPrefilter=1#cmp-menu-container'

def keywordsFileName(listId):
    if listId == 1:
        return 'frequency.csv'
    elif listId == 2:
        return 'tfIdfWords.csv'
    else:
        return 'tfIdfWords.csv'

def scrape(postingUrl,bank):
    print(postingUrl)
    global count
    headers = ["Job No", "Institution", "List Id {1,2,3}", "URL (URL of the job posting)"]

    values = [count, "PNC Bank", "2", postingUrl]
    
    indices = list(range(1, 101))
    
    wordCountDict = dict.fromkeys(indices,0)
    postingDict = dict(zip(headers, values))
    
    enc = 'utf-8'
    with open("Data/frequency_final.csv", 'r', encoding = enc) as f:
        reader = csv.reader(f)
        keywords = list(reader)
    arr = [i[0] for i in keywords]
    
    try:
        hdr = {'User-Agent': 'Mozilla/5.0'}
        request = urllib2.Request(postingUrl,headers=hdr)
        html_page = urlopen(request)
        soup2 =  BeautifulSoup(html_page,"html.parser")
        if bank == 'PNC':
            value = soup2.find('input', {'id':'preLoadJSON'})
            for word in value.get('value').split('"CandLandPageText"')[1].replace("\\u003", " ").split():
                if(word in arr):
                    ind = arr.index(word)
                    wordCountDict[ind] = wordCountDict.get(ind, 0) + 1
            dataDict = postingDict.copy()  
            return {**postingDict, **wordCountDict}
                
        else:
            value = soup2.findAll("div", {"class": "desc"})
            for val in value:
                st = val.text.split()
                for word in st:
                    if(word in arr):
                        ind = arr.index(word)
                        wordCountDict[ind] = wordCountDict.get(ind, 0) + 1
    except:
        print("Exception")
        return ("An exception occurred")
    
    count = count + 1

    dataDict = postingDict.copy()  
    return {**postingDict, **wordCountDict} 
 
def scrape_page_pnc(page):
    response = urlopen(page)
    bs_obj = BeautifulSoup(response, "html.parser")
    if bs_obj:
        links = bs_obj.findAll('div',{'class': 'cmp-section cmp-with-border'})
        for page in links:
            linkTags = page.find_all('a', attrs={'href': re.compile("/cmp/_")})
            for tag in linkTags:
                link = tag.get('href')
                postingUrl = BASE + link
                pageDict = scrape(postingUrl,"PNC")
                if (type(pageDict) is dict):
                        data = pd.DataFrame([pageDict])
                        # if file does not exist write header 
                        if not os.path.isfile('pnc_freq.csv'):
                            data.to_csv('pnc_freq.csv', header='column_names')
                        else: 
                            # else it exists so append without writing the header
                            data.to_csv('pnc_freq.csv', mode='a', header=False)
                else:
                    print("exception?")
    


def scrape_pages_schwab(pages):
    stateList = {}
    for p in pages:
        pageHtml = urlopen(p)
        soup = BeautifulSoup(pageHtml)
        table = soup.find('table', 'JobListTable')
        pageDataframe = pd.DataFrame([])
        for link in table.findAll('a', attrs={'href': re.compile("/ShowJob")}):
            page = Schwab_BASE + link.get('href')
            state = table.findAll("td",{"class": "colstate"})
            for s in state:
                st = re.sub(r'[^ \w\.]', '', s.text)
                if st not in stateList:
                    stateList[st] = 1
                else:
                    stateList[st] += 1
            pageDict = scrape(page,"Schwab")
            if (type(pageDict) is dict):
                data = pd.DataFrame([pageDict])
                # if file does not exist write header 
                if not os.path.isfile('schwabData_tr.csv'):
                    print("writing to csv")
                    data.to_csv('schwabData_tr.csv', header='column_names')
                else: # else it exists so append without writing the header
                    data.to_csv('schwabData_tr.csv', mode='a', header=False)
    with open('states.csv', 'a+', encoding = 'utf-8') as f: 
        for key in stateList.keys():
            f.write("%s,%d\n"%(key,stateList[key]))
        

def get_pages(categories):
    pageLinks = []
    for key, value in categories.items():
        if("---" not in key):
            if(value > 30):
                pageCount = 1
                for i in range(1,value,29):
                    pageLinks.append(Schwab_BASE + key + "Page-" + str(pageCount))
                    pageCount = pageCount + 1
                
            else:
                pageLinks.append(Schwab_BASE + key)
    return pageLinks

def get_category(response):
    bs_obj = BeautifulSoup(response, "html.parser")
    if bs_obj:
        cat_table = bs_obj.find_all("table",{"class": "list-categores-table"})

        catLinks = []
        numPages = []
        cat = []

        for page in cat_table:
            linkTags = page.find_all('a', attrs={'href': re.compile("/ListJobs")})
            divTags = page.find_all('div')
            for tag in linkTags:
                catLinks.append(tag.get('href'))
                cat.append(tag.text)
            for div in divTags:
                num = int(re.search(r'\d+', div.text).group())
                numPages.append(num)
        #categCsv = dict(zip(cat, numPages))
        #plt.bar(range(len(categCsv)), list(categCsv.values()), align='center')
        #plt.xticks(range(len(categCsv)), list(categCsv.keys()))
        #plt.xticks(rotation = 90)
        #plt.imshow()
        return dict(zip(catLinks, numPages))

def scrape_site(Url):
    scrape_page_pnc(FIRST_LISTING_PAGE)
    for i in range(12,996,12):
        nextPageUrl = LISTING_URL_PREFIX +  str(i) + "&" + LISTING_URL_POSTFIX
        scrape_page_pnc(nextPageUrl)

def get_data(bank,Url):
    if (bank == "PNC"):
        scrape_site(Url)
    elif (bank == "Schwab"):
        res = urlopen(Url)
        categories = get_category(res)
        pages = get_pages(categories)
        scrape_pages_schwab(pages)

if __name__ == '__main__':
    get_data("PNC","https://www.indeed.com/jobs?q=Pnc+Bank")

https://www.indeed.com//cmp/_/job?jk=f5df4ed098b9df5e&tk=1d3k5cba85n8h800
https://www.indeed.com//cmp/_/job?jk=151be1068dfc5659&tk=1d3k5cba85n8h800&rd=n5r6n1S2hixwO8gpEj_fuBcDY5Ws-Xs_yuMBKiKofo91uS6Q69ktC-y-BiMHNS3ifTU3X4Uxq_eODsJbhaKG6pWY1xKe9vZtztShvwGdXaerH6qUBDSsRS62g--BFrzKouDvw7C99PFbmkrT3QFP0uWYzf8t2v48xYPf5UDApcKXrCa9JQN6mCzgF-ipsta3fPsj52la9AwjMQVWl682JdgUcJATQurb1BJepSRpIs7YSHKsZCMP7qrB6wYcQl1cKMnZkfPNyJbL00eZxmBrY4hFSsKRz8QXoeFuZNX1Ainmk2qRYzQbfQeB_ACea33sMetmhMddIY2eLOpR1B-w7oyiNuSBenn-Hr9zPz-vLv91sYrtp4UMHrAvNO2o-lgW1yMWjgEBEYW163m5OygQJ37Ied0U4RW5VWBwilQuyfjDRr9iafauCdAMs-a7nQ4Ka4mBPPU_EKSuoN7GAShdO4KYM8zPZOY-Q6hllosYCVL0K7v8chWXrtZrMqRFDkCi6wJIt8BtV4T20P_Mg1MDGVhPEW2caNJXaIJgBBCxC_9uh3iPZf7ArbnVpZ4BOPPyg8kWxdjTvwu8c9ChIbhWBwZ_nmzVWXO7oQ57N6ye20M
https://www.indeed.com//cmp/_/job?jk=5f0e0ffe66f32922&tk=1d3k5cba85n8h800
https://www.indeed.com//cmp/_/job?jk=faad03cff940f3af&tk=1d3k5cba85n8h800&rd=n5r6n1S2hixwO8gpEj_fuBcDY5Ws-Xs_yuMBKiKofo91uS6Q69ktC-y-BiMHNS3ifTU3X4Uxq_eODs

https://www.indeed.com//cmp/_/job?jk=4863a5c07b0714ae&tk=1d3k5o3705nci800&rd=n5r6n1S2hixwO8gpEj_fuBcDY5Ws-Xs_yuMBKiKofo91uS6Q69ktC-y-BiMHNS3ifTU3X4Uxq_eODsJbhaKG6pWY1xKe9vZtztShvwGdXae_PdvIyhuetuoqVUCCo8KwgiYzi6EK7Pwj1LWox-Q0WxanAS90XilHtdYfaMmsJhNYxMvQA33IFstx9T3T0xgTdFzi0US1cJlW74wIa0lPDgApmnpSE35zONxTTCHT6xLv41D1aZG-X7Xo_yp3xC7CEz3rKPdKCGz25Cws6vuwpsc7OyfVnKmXGo1kYaVrrgCtyLU-OSuDhC5iOiK88xd5BM1uVtIbD5AMAGXo8PoENXQm0ZHt932Go_GO3X_tXrE_AM2Q37o19nYYykB-7kkfHm6-bzceELI2yQcJw1_aH3M1kfaoAjbdrs8QhxokhWM04zSe2SKPc9nE3dBFkyO36nqaZQWTVW6ipa0gbhNRzfCTYn8glfL2vCeHQrL6-8NnYEf3c3nbYZ48ZimXxhTDABe6oLfHQ7t409g-mO9skGUXNoSw_trnJ4Im3HG4H0yypJzJz-J-MZ9ckDADrz4I2BUToRV_uWeP2k3pAftnzbQfP0S3Gpe1vVKWrzHekRY
https://www.indeed.com//cmp/_/job?jk=a41f463cd326916f&tk=1d3k5o3705nci800&rd=n5r6n1S2hixwO8gpEj_fuBcDY5Ws-Xs_yuMBKiKofo91uS6Q69ktC-y-BiMHNS3ifTU3X4Uxq_eODsJbhaKG6pWY1xKe9vZtztShvwGdXaccKg4T8vk10KhLQSH7LDNuIfspErDWnLnr4kW8YhCoJWdbNcY6-dmJU3NpHNJq04aHx3_GSVOAIL8CmLwokAwE0VaRJcEO55PkDiF3QaSjkatryGZCsepDYc

https://www.indeed.com//cmp/_/job?jk=3f87bab36d1ffa40&tk=1d3k6564o5nci800
https://www.indeed.com//cmp/_/job?jk=01158f625f75c6fe&tk=1d3k6564o5nci800&rd=n5r6n1S2hixwO8gpEj_fuBcDY5Ws-Xs_yuMBKiKofo91uS6Q69ktC-y-BiMHNS3ifTU3X4Uxq_eODsJbhaKG6pWY1xKe9vZtztShvwGdXad9kw89DMYy0FsqbZVS8FfH0QMrxhZDXEISucCJeWhviKq24H_hibhvjJw5uTl16V6TZb4NJTDBmMHnBhEzcZZdZc0H-ei0NPFJm3_a-vpqXt2-f-0MhWCWO6vGktKfkUPXRmNaHdqO_Xjk8Zw0tr6TC9HR9B5lLixyIIVxA92fmEWDai9qJMXIIB2OOS3LIT8-HoYqZwoocp80eAekekJrlTwFUkgW_VOiPGPIuUz6qEIwQDfBI6kSh_a1_tj8av1tzszKxv45WukXphmV4HbFX4TY_35_M41sVFFlvAhJYBbFMGA7vuKjPt6lfpGWhMsAaWCE8PokxCFTeW-VO93nRte89I4IkrihbLSoDNacXsmzZEWe5ArS9ydTBVGV53CAH74ylAwNyhCOTdOTGeIDEgHCtLzHZTNZ3BPg9a0zqkRpbTI0p_pNsaIAaECOOIOKsNw3A0m2fUQ7-5oWWWNP1dLxgxZLFaqhqY8oqcZsaAbyiyEOnW3rKZBqjoUFeAg
https://www.indeed.com//cmp/_/job?jk=14c67eb2f029e592&tk=1d3k6564o5nci800
https://www.indeed.com//cmp/_/job?jk=8e6adbfc6fa30f9a&tk=1d3k6564o5nci800
https://www.indeed.com//cmp/_/job?jk=a33a8b6b233d9a5c&tk=1d3k6q015ak4j801
Excepti

https://www.indeed.com//cmp/_/job?jk=9834c89743fa17ad&tk=1d3k70pn45n8h800&rd=n5r6n1S2hixwO8gpEj_fuBcDY5Ws-Xs_yuMBKiKofo91uS6Q69ktC-y-BiMHNS3ifTU3X4Uxq_eODsJbhaKG6pWY1xKe9vZtztShvwGdXadGCJxFukyqLr5JzSUX9GI9oaNVWMnMIuHT-i0nWNwlP_9tsYWcFeJuQvV2WT3_rlb2rnstuyfsWpR36fF7Vb-mqENloxaEd3RV6Idc9zWgQCRASt_xuCqZlYcwH-3-eGc9jXrkNiK11cbANuSBj455Nx2tKX8mrlYHyXQvGdD06V_2sKf2qajrQdXzcWG-yQxqUcv9uCbDtwB2YXYTIKoJ8a2SWpLWjw3SHsbpXITUBnmT-41L7sVtExngQOpsNfa4htWKICdpziyBqn8r60SAzD5v_-9RA8u-jrJOwaMrI6HF47f4PLq5fTDUkWb8kuHfOQq_PfDTuGw9q1dwsI4mBjGHQb0QY1FHgtLrjXYO7pp-WswiaDiN5D1vLr4iYBWW0Ci28LLeMNu0PbgTKZYfvCOroaW0hxjVhnHXbitejgeyafeQbo0Vp4mBVP60sprIY0Ar79H64AJHJh7y8wdRzmh-uC0gwDyNDtZvkTK-3QZI0WAypaRbFTDksNwFYkU
https://www.indeed.com//cmp/_/job?jk=3d354eb27bf1238b&tk=1d3k70pn45n8h800
https://www.indeed.com//cmp/_/job?jk=ae60ebf38834dd2f&tk=1d3k70pn45n8h800
https://www.indeed.com//cmp/_/job?jk=4ef6c1d8a99472f3&tk=1d3k70pn45n8h800&rd=n5r6n1S2hixwO8gpEj_fuBcDY5Ws-Xs_yuMBKiKofo91uS6Q69ktC-y-BiMHNS3ifTU3X4Uxq_eODs

https://www.indeed.com//cmp/_/job?jk=561b4c2bcd56c859&tk=1d3k7ck4jakci800
https://www.indeed.com//cmp/_/job?jk=5a26faec22bc6e3a&tk=1d3k7ck4jakci800&rd=n5r6n1S2hixwO8gpEj_fuBcDY5Ws-Xs_yuMBKiKofo91uS6Q69ktC-y-BiMHNS3ifTU3X4Uxq_eODsJbhaKG6pWY1xKe9vZtztShvwGdXafeDn8diMPE6R4BmHqHF_COctG6BrPQvfQqdKL_TFbvPvuaTetBuyqqVl3_CF2_qSHWap75ln_dtgzCmLZbF9kiDWn7-0HTTsII5o0k95EhWODvYK3iVs13OvD-M9O4HLuCC0xQQF8SadJsrIrhwCpHavMvvj_Lu93WSA0FMozMYaR52etpBzAShLoCRg7BuAW1jaBdulAEB_j7UBTgLXqAtdqI_BGNwJ_61vzGczmIkBzejL6KNBzk4lveedRbEF2mvgG4Z1pJP_D5hKDxCJDb6fOIoJU8GTH5nvTze4mLszF-qzbZTj77VRBeLcHw7uAoRzqseuElrgoPbYZSWy36uP-Rbx2Ot0nVJ6Chy4eUQR1h5Dqvyp9ELvrflYrICI8P_dGSKLwu4v2NeEZ0H66rPFJSDAOEq3txsUEHJaQzyQFIrKNP8d6u8vMhwM4FGxhkZ471uaFfaXX5BmWTSlVjgYPN4cpl0q4O2hGNXxXo9iH4rlOXJHuPEEo1iirzoCs
https://www.indeed.com//cmp/_/job?jk=416d7e4493d5c659&tk=1d3k7ck4jakci800
https://www.indeed.com//cmp/_/job?jk=2e0daada64edaf48&tk=1d3k7ck4jakci800&rd=n5r6n1S2hixwO8gpEj_fuBcDY5Ws-Xs_yuMBKiKofo91uS6Q69ktC-y-BiMHNS3ifTU3X4Uxq_eODs